# Chapter 18. 나이브 베이즈

## 18.0 소개

* 베이즈 이론

$$P(A|B) = {{P(B|A) P(A)}\over{P(B)}}$$

* 머신러닝에서 베이즈 이론을 분류에 적용한 것이 나이브 베이즈 분류기 이다.
    * 실용적 머신러닝에서 필요한 기능을 제공한다.
    * 직관적이고, 적은 양의 데이터에 사용 가능하고, 훈련과 예측 계산 비용이 적게 들고, 환경이 변해도 안정적인 결과를 보여준다.

## 18.1 연속적인 특성으로 분류기 훈련하기
* 사이킷런의 가우시안 나이브 베이즈 분류기 사용

In [2]:
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB

iris =datasets.load_iris()
features = iris.data
target = iris.target

classifier = GaussianNB()
model =classifier.fit(features,target)

In [3]:
new_observation = [[4, 4, 4, 0.4]]

model.predict(new_observation)

array([1])

* 가우시안 나이브 베이즈 : 정규분포를 가정하므로, 연속적인 특성인 경우 잘 맞는다.

* 나이브 베이즈 분류기 : 타깃 클래스에 사전 확률을 지정할 수 있다.
    * GaussianNB 클래스의 priors 파라미터를 활용하면, 타깃 벡터의 각 클래스에 할당할 확률 리스트 전달 가능하다.

In [4]:
clf = GaussianNB(priors=[0.25, 0.25, 0.5])

model = classifier.fit(features, target)

In [5]:
model.predict(new_observation)

array([1])

## 18.2 이산적인 카운트 특성으로 분류기 훈련하기
* 다항(multi-nomial) 나이브 베이즈 분류기 사용

In [7]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

text_data = np.array(['I love Brazil. Brazil',
'Brazil is best',
'Germany beats both'])

# BoW 제작
count = CountVectorizer()
bag_of_words = count.fit_transform(text_data)

# 특성 행렬 제작
features = bag_of_words.toarray()

# 타깃 벡터 제작
target = np.array([0,0,1])

# 각 클래스별 사전 확률을 지정한 다항 나이브 베이즈 객체 제작
classifier = MultinomialNB(class_prior=[0.25, 0.5])

# 모델 훈련하기
model =classifier.fit(features, target)

* 다항 나이브 베이즈 : 특성이 다항 분포로 가정, 실전에서 많이 사용한다.
    * 텍스트 분류에서 활용도가 높다.

In [10]:
# 새로운 샘플을 만든다.
new_observation = [[0,0,0,1,0,1,0]] 

model.predict(new_observation)

array([0])

## 18.3 이진 특성으로 나이브 베이즈 분류기 훈련하기

* 이진 데이터가 있고 나이브 베이즈 분류기로 훈련한다.
* 베르누이 나이브 베이즈 분류기 사용한다.

In [11]:
import numpy as np
from sklearn.naive_bayes import BernoulliNB

features = np.random.randint(2, size=(100, 3))

target = np.random.randint(2, size=(100, 1)).ravel()

classifier = BernoulliNB(class_prior=[0.25, 0.5])

model = classifier.fit(features, target)

* 베르누아 나이브 베이즈 분류기 : 모든 특성이 두 종류의 값ㅁ나 발생할 수 있는 이진 특성이라 가정한다.
    * 원-핫 인코딩된 순서가 없는 범주형 특성
    * 다항 나이브 베이즈와 비슷하게 베르누이 나이브 베이즈도 텍스트 분류에 많이 사용된다.
    * 한 문서에 어떤 단어가 등장하는지 여부를 담는다.
    * 균등 분포를 사용하려면, fit_prior = False 로 지정한다.

In [12]:
model_uniform_prior = BernoulliNB(class_prior=None, fit_prior=True)

## 18.4 예측 확률 보정하기
* 나이브 베이즈 분류기의 예측확률을 이해가 쉽게 수정한다.
* CalibratedClassifierCV 활용한다.

In [13]:
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV

iris = datasets.load_iris()
features = iris.data
target = iris.target

# 가우시안 나이브 베이즈 객체 제작
classifier = GaussianNB()

# 시그모이드 보정을 사용해 보정 교차 검증 제작
classifier_sigmoid = CalibratedClassifierCV(classifier, cv=2, method='sigmoid')

# 확률 보정
classifier_sigmoid.fit(features, target)

# 새로운 샘플 제작
new_observation = [[2.6,2.6,2.6,0.4]]

# 보정된 확률 확인
classifier_sigmoid.predict_proba(new_observation)

array([[0.31859969, 0.63663466, 0.04476565]])

* 클래스 확률은 머신러닝 모델에서 유용하게 사용된다.
    * 사이킷런의 대부분 학습 알고리즘 : predict_proba 메서드로 예측된 클래스 소속 확률을 확인할 수 있다.
    * 나이브 베이즈는 클래스에 대한 예측 확률의 순위는 유요하다. 그러나 예측 확률이 0 혹은 1 극단적으로 가까워지는 경향이 있다.
* 의미 있는 예측 확률을 위해 보정(calibration) 작업 필요하다.
    * 반환된 예측 확률은 k-폴드의 평균이 된다.
* 가실제 확률을 예측한 예를 든다.

In [14]:
# 가우시안 나이브 베이즈를 훈련하고 클래스 확률을 예측한다.
classifier.fit(features, target).predict_proba(new_observation)

array([[2.31548432e-04, 9.99768128e-01, 3.23532277e-07]])

In [15]:
# 보정된 예측 확률 확인하면 매우 다른 결과임을 안다.
classifier_sigmoid.predict_proba(new_observation)

array([[0.31859969, 0.63663466, 0.04476565]])

* CalibratedClassifierCV : method 파라미터에 2 개의 보정 방법을 지원한다.
    * 플랫의 시그모이드 모델과 등위회귀
    * 등위회귀 : 비모수 모델이므로 샘플 크기가 작으면 과적합 경향이 있다.
    * 여기서는 150개 샘플을 가진 데이터셋이므로 플랫의 시그모이드 모델을 사용한다.